In [8]:
WORKER_COUNT = 24

In [9]:
# whether to log each feature and sequence status
verbose = True

In [10]:
import gc
import os
import pandas as pd
pd.options.display.max_rows = 5000
import numpy as np
import json
import datetime
import matplotlib.pyplot as plt
import itertools
import sys
sys.path.append('..')

In [11]:
# setup paths
pwd = os.getcwd().replace("notebooks","")
path_cache = pwd + 'cache/'
path_data = pwd + 'data/'

In [12]:
# setup logging
# any explicit log messages or uncaught errors to stdout and file /logs.log
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s",
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs")),
        logging.StreamHandler()
    ])
# init logger
logger = logging.getLogger()
# make logger aware of any uncaught exceptions
def handle_exception(exc_type, exc_value, exc_traceback):
    if issubclass(exc_type, KeyboardInterrupt):
        sys.__excepthook__(exc_type, exc_value, exc_traceback)
        return

    logger.error("Uncaught exception", exc_info=(exc_type, exc_value, exc_traceback))
sys.excepthook = handle_exception

In [13]:
from deepvideoclassification.pretrained_CNNs import pretrained_model_names, pretrained_model_names_bucketed


# Create list of experiments to be run

* batch 1 = run frozen image MLP, LRCNNs and concat models on 1 of each pretrained_model_name in buckets (bucketed on feature sizes and limited to max sequence_length of 10)

* batch 2 = for best configurations from batch 1, run other pretrained models in buckets and run longer sequence lengths, maybe try different convolution kernel sizes

* batch 3 = run trainable MLP and LRCNN on best performing frozen variants

* batch 4 = run trainable but initializing with best CNN weights

* batch 5 = run C3D models

* batch 6 = analyze effect of dropout and pooling with best model

# Batch 1

In [ ]:
experiment_batch_name = 'experiment_batch_1'

In [ ]:
# init model id - need to make sure we pick up where we leave off don't overwrite it between batches
model_id_start = 0

In [ ]:
# init list of experiments
experiments = []

In [ ]:
pooling = 'max'
layer_sizes = [512, 256, 128, 0]
dropouts = [0.2]
sequence_lengths = [3,5,10]
sequence_models = ["LSTM", "SimpleRNN", "GRU", "Convolution1D"]
sequence_model_layer_counts = [1,2]

In [ ]:
####################
### image_MLP_frozen 
####################

for pretrained_model_name in pretrained_model_names_bucketed:
    for layer_1_size in layer_sizes:
        for layer_2_size in layer_sizes:
            for layer_3_size in layer_sizes:
                for dropout in dropouts:

                    # build experiment parameters
                    experiment = {}
                    
                    experiment['architecture'] = 'image_MLP_frozen'
                    experiment['sequence_length'] = 1
                    experiment['pretrained_model_name'] = pretrained_model_name
                    experiment['layer_1_size'] = layer_1_size
                    experiment['layer_2_size'] = layer_2_size
                    experiment['layer_3_size'] = layer_3_size
                    experiment['dropout'] = dropout
                    experiment['pooling'] = 'max' # outperforms avg across all parameters
                    
                    # add to list of experiments
                    experiments.append(experiment)

In [ ]:
####################
### video_MLP_concat
####################

for sequence_length in sequence_lengths:
    for pretrained_model_name in pretrained_model_names_bucketed:
        for layer_1_size in layer_sizes:
            for layer_2_size in layer_sizes:
                for layer_3_size in layer_sizes:
                    for dropout in dropouts:

                        # build experiment parameters
                        experiment = {}

                        experiment['architecture'] = 'video_MLP_concat'
                        experiment['pretrained_model_name'] = pretrained_model_name
                        experiment['layer_1_size'] = layer_1_size
                        experiment['layer_2_size'] = layer_2_size
                        experiment['layer_3_size'] = layer_3_size
                        experiment['dropout'] = dropout
                        experiment['pooling'] = 'max' # outperforms avg across all parameters
                        experiment['sequence_length'] = sequence_length

                        # add to list of experiments
                        experiments.append(experiment)

In [ ]:
######################
### video_LRCNN_frozen
######################

for sequence_length in sequence_lengths:
    for pretrained_model_name in pretrained_model_names_bucketed:
        for layer_1_size in layer_sizes:
            for layer_2_size in layer_sizes:
                for layer_3_size in layer_sizes:
                    for dropout in dropouts:
                        for sequence_model in sequence_models:
                            for sequence_model_layers in sequence_model_layer_counts:

                                # build experiment parameters
                                experiment = {}

                                experiment['architecture'] = 'video_LRCNN_frozen'
                                experiment['pretrained_model_name'] = pretrained_model_name
                                experiment['layer_1_size'] = layer_1_size
                                experiment['layer_2_size'] = layer_2_size
                                experiment['layer_3_size'] = layer_3_size
                                experiment['dropout'] = dropout
                                experiment['pooling'] = 'max' # outperforms avg across all parameters
                                experiment['sequence_model'] = sequence_model
                                experiment['sequence_model_layers'] = sequence_model_layers
                                experiment['sequence_length'] = sequence_length

                                # add to list of experiments
                                experiments.append(experiment)

In [ ]:
########################
### convert to dataframe
########################

experiments = pd.DataFrame(experiments)

### create model id column for this experiment batch
model_id_list = list(range(0,len(experiments)))
experiments['model_id'] = model_id_list

# assign to workers
experiments['WORKER'] = experiments['model_id'].apply(lambda x: x % WORKER_COUNT)

In [ ]:
experiments.shape

In [ ]:
############################################
### remove invalid experiment configurations
############################################

# Just won't run experiments for those model_ids - not an error that model ids not congituous count from 0!

# delete video experiments with 0 neurons in a layer with nonzero neurons in later layers
experiments = experiments[~((experiments['layer_1_size'] == 0) & (experiments['layer_2_size'] > 0))]
experiments = experiments[~((experiments['layer_1_size'] == 0) & (experiments['layer_3_size'] > 0))]
experiments = experiments[~((experiments['layer_2_size'] == 0) & (experiments['layer_3_size'] > 0))]

# delete video experiments where convolution_kernel_size > sequence_length (convolution_kernel_size defaults to 3 and not set in this batch)
experiments = experiments[~((experiments['sequence_model'] == 'Convolution1D') & (experiments['sequence_length']<=3))]

In [ ]:
# delete LRCNN_frozen experiments with layer_1_size == 0
experiments = experiments[~((experiments['architecture'] == 'video_LRCNN_frozen') & (experiments['layer_1_size']==0))]

In [ ]:
##################################
### output experiment batch to CSV
##################################
print(experiment_batch_name)
experiments.to_csv(pwd + "experiments/" + experiment_batch_name + '.csv', index=False)

In [ ]:
print(experiments.shape)
experiments.tail().T

In [ ]:
# upload to s3
response = os.system("aws s3 cp " + pwd + "experiments/" + experiment_batch_name + '.csv s3://thesisvids/penguins/' + experiment_batch_name + '.csv')
if response == 0:
    print("upload success")
else:
    print("upload error")

In [ ]:
!aws s3 ls s3://thesisvids/penguins/

# Batch 2

In [ ]:
# run other pretrained models for best configurations from batch 1
# and run longer sequence lengths

In [ ]:
experiment_batch_name = 'experiment_batch_2'

In [ ]:
# init model id - need to make sure we pick up where we leave off don't overwrite it between batches
model_id_start = pd.read_csv(pwd + "experiments/experiment_batch_1.csv")['model_id'].max() + 1

In [ ]:
# init list of experiments
experiments = []

In [ ]:
# TODO

In [ ]:
########################
### convert to dataframe
########################

experiments = pd.DataFrame(experiments)

### create model id column for this experiment batch
model_id_list = list(range(0,len(experiments)))
experiments['model_id'] = model_id_list

# assign to workers
experiments['WORKER'] = experiments['model_id'].apply(lambda x: x % WORKER_COUNT)

In [ ]:
##################################
### output experiment batch to CSV
##################################
print(experiment_batch_name)
experiments.to_csv(pwd + "experiments/" + experiment_batch_name + '.csv', index=False)

# Batch 3

In [ ]:
# run trainable MLP and LRCNN on best performing frozen variants

In [ ]:
#######################
### image_MLP_trainable
#######################

architecture = 'video_LRCNN_trainable'

In [ ]:
#########################
### video_LRCNN_trainable
#########################

architecture = 'video_LRCNN_trainable'

# Batch 4

In [ ]:
# run trainable but initializing with best CNN weights

# Batch 5

# Analyze results

## load results.json for all models into dataframe

In [ ]:
path_models = pwd + 'models/'

results = []

for folder, subs, files in os.walk(path_models):
    for filename in files:
        if 'results.json' in filename:
            with open(os.path.abspath(os.path.join(folder, filename))) as f:
                data = json.load(f)
            results.append(data)

results = pd.DataFrame(results)        
results.sort_values("fit_val_acc", inplace=True, ascending=False)

In [ ]:
results.head(10).T

## Merge done status onto experiments

In [ ]:
experiments = pd.merge(experiments, results[['model_id','fit_val_acc']], left_on='model_id', right_on='model_id', how='left')
experiments['done'] = (experiments['fit_val_acc']>0).astype(int)
del experiments['fit_val_acc']

In [ ]:
experiments.head()

In [ ]:
print("{}/{} experiments done".format(experiments[experiments['done'] == 1].shape[0], len(experiments)))

## total experiments, split by architecture

In [ ]:
experiments['architecture'].value_counts()

## analyze remaining experiments, split on architecture

In [ ]:
experiments[experiments['done']==0]['architecture'].value_counts()

In [ ]:
experiments[experiments['architecture'] == 'video_MLP_concat']

In [ ]:
experiments[experiments['architecture'] == 'video_MLP_concat'].sort_values("sequence_length").head()

# Copy experiment files to s3

In [ ]:
model_ids = list(results['model_id'])
model_ids.sort()

In [ ]:
for i, model_id in enumerate(model_ids):
    
    path_model = pwd + '/models/' + str(model_id) + '/'

    # aws s3 ls on path returns 0 if it exists so check if doesn't exist, then sync
    if os.system("aws s3 ls s3://thesisvids/penguins/models/" + str(model_id) + "/") > 0:
        print("Synching {}/{} - model_id={}".format(i+1,len(model_ids),model_id))
        response = os.system("aws s3 sync " + path_model + " s3://thesisvids/penguins/models/" + str(model_id) + "/")
        if response != 0:
            print("ERROR syncing model_id = {}".format(model_id))
    else:
        print("Already synched {}/{} - model_id={}".format(i+1,len(model_ids),model_id))

# Copy experiment files from s3

In [ ]:
# TODO 
# FIRST SYNCH TO S3 THEN DELETE MODELS FOLDER AND SYNC FROM S3

# Debug experiment worker

In [14]:
from deepvideoclassification.architectures import Architecture

In [15]:
WORKER_ID = 0
GPU_ID = 7
experiment_batch_name = 'experiment_batch_1'

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=str(WORKER_ID)

In [16]:
# setup logging
# separate log file for each worker
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s, [%(levelname)-8s] [%(filename)s:%(lineno)d] %(message)s',
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs_" + str(WORKER_ID))),
        logging.StreamHandler()
    ])
# init logger - will pass this to our architecture
logger = logging.getLogger()

logger.info("Start worker {} (GPU={}) processing {}".format(WORKER_ID, GPU_ID, experiment_batch_name))

2019-01-21 17:28:14,285 [MainThread  ] [INFO ]  Start worker 0 (GPU=7) processing experiment_batch_1


In [17]:
# load list of experiments
experiments = pd.read_csv(pwd + "experiments/" + experiment_batch_name + '.csv')

In [19]:
experiments[experiments['model_id'].isin([2359,2391, 2447])]

,architecture,dropout,layer_1_size,layer_2_size,layer_3_size,pooling,pretrained_model_name,sequence_length,sequence_model,sequence_model_layers,model_id,WORKER
1237,video_LRCNN_frozen,0.2,512,256,128,max,inception_resnet_v2,5,Convolution1D,2.0,2359,7
1269,video_LRCNN_frozen,0.2,512,128,128,max,inception_resnet_v2,5,Convolution1D,2.0,2391,15
1301,video_LRCNN_frozen,0.2,256,512,256,max,inception_resnet_v2,5,Convolution1D,2.0,2447,23


In [20]:
# for row in experiments.values:
debug_model_id = 2339

row = list(experiments[experiments['model_id'] == debug_model_id].values[0])

# get experiment params from dataframe row
experiment = dict(zip(experiments.columns, row))

In [22]:
experiment

{'WORKER': 11,
 'architecture': 'video_LRCNN_frozen',
 'dropout': 0.2,
 'layer_1_size': 512,
 'layer_2_size': 256,
 'layer_3_size': 512,
 'model_id': 2339,
 'pooling': 'max',
 'pretrained_model_name': 'inception_resnet_v2',
 'sequence_length': 5,
 'sequence_model': 'SimpleRNN',
 'sequence_model_layers': 2.0}

In [23]:
experiments[experiments['model_id'] == debug_model_id].T

,1217
architecture,video_LRCNN_frozen
dropout,0.2
layer_1_size,512
layer_2_size,256
layer_3_size,512
pooling,max
pretrained_model_name,inception_resnet_v2
sequence_length,5
sequence_model,SimpleRNN
sequence_model_layers,2


In [24]:
print(str(experiment["model_id"]) + "   " + "X"*60)
logging.info("Begin experiment for model_id={} on GPU:{} ".format(experiment['model_id'], os.environ["CUDA_VISIBLE_DEVICES"]))
print(experiment)

architecture = Architecture(model_id = experiment['model_id'], 
                            architecture = experiment['architecture'], 
                            sequence_length = experiment['sequence_length'], 
                            pretrained_model_name = experiment['pretrained_model_name'],
                            pooling = experiment['pooling'],
                            sequence_model = experiment['sequence_model'],
                            sequence_model_layers = experiment['sequence_model_layers'],
                            layer_1_size = experiment['layer_1_size'],
                            layer_2_size = experiment['layer_2_size'],
                            layer_3_size = experiment['layer_3_size'],
                            dropout = experiment['dropout'],
                            verbose=True)

2019-01-21 17:29:09,673 [MainThread  ] [INFO ]  Begin experiment for model_id=2339 on GPU:0 
2019-01-21 17:29:09,676 [MainThread  ] [INFO ]  Loading data


2339   XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
{'sequence_length': 5, 'sequence_model_layers': 2.0, 'WORKER': 11, 'layer_1_size': 512, 'architecture': 'video_LRCNN_frozen', 'sequence_model': 'SimpleRNN', 'dropout': 0.2, 'model_id': 2339, 'layer_3_size': 512, 'pretrained_model_name': 'inception_resnet_v2', 'pooling': 'max', 'layer_2_size': 256}


2019-01-21 17:29:10,639 [MainThread  ] [INFO ]  Features already cached: /mnt/seals/cache/features/inception_resnet_v2/max/
2019-01-21 17:29:10,640 [MainThread  ] [INFO ]  Loading features sequence data into memory [may take a few minutes]


Done initializing data with #samples: train=60509, valid=6408, test=3135


ValueError: Input 0 is incompatible with layer flatten_2: expected min_ndim=3, found ndim=2

In [16]:
architecture.train_model()

Train on 60509 samples, validate on 6408 samples
Epoch 1/20
15552/60509 [======>.......................] - ETA: 36s - loss: 4.1813 - acc: 0.7388

KeyboardInterrupt: 